In [3]:
import h5py
import torch

In [1]:
import sys
import importlib.util

solver_dir = "/home/ubuntu/SML/solver"

sys.path.append(solver_dir)

file_path = "/home/ubuntu/SML/solver/solver.py"
spec = importlib.util.spec_from_file_location("solver", file_path)
solver = importlib.util.module_from_spec(spec)
spec.loader.exec_module(solver)

In [2]:
E = 113.8e9
nu = 0.342
rho = 4.47e-3

In [ ]:
i = 0
file_path = f'/data/SimJEB/SimJEB_fixed/sample_{i}.h5'


with h5py.File(file_path, 'r') as h5_file:
    forces = torch.tensor(h5_file['forces'][:])
    rbe2 = torch.tensor(h5_file['rbe2'][:])
    rbe3 = torch.tensor(h5_file['rbe3'][:])
    outputs = torch.tensor(h5_file['outputs'][:])


coords, elements = solver.vtk_loader_to_torch(f'/data/SimJEB/vtk/{i}.vtk')

rbe2_mask = torch.zeros(coords.shape[0], dtype=torch.int32)
rbe2_mask[rbe2] = 1

print('coords', coords.shape, 'elements', elements.shape, 'rbe2 mask', rbe2_mask.shape, 'rbe2', rbe2.shape, 'rbe3', rbe3.shape, 'outputs', outputs.shape)


2025-02-11 16:56:12.713 (1354.201s) [    7F0C6F29E740]      vtkDataReader.cxx:1507  WARN| Error reading binary data!


AttributeError: 'NoneType' object has no attribute 'GetData'

In [4]:
K = solver.compute_L_matrix(coords, elements, E, nu, device="cuda:0")
F_target = torch.zeros([coords.shape[0], 6], dtype=torch.float64)
f = solver.compute_nodal_forces(K, elements, outputs[:,0,:3], device="cpu")[rbe3[0]][-1]
F_target[rbe3] = torch.tensor([0,0,f],dtype=torch.float64)

참값의 오차

In [5]:
F = compute_nodal_forces(K, elements, outputs[:,0,:3], device="cpu")
residual = torch.norm(F - F_target, dim=-1)
residual[rbe2] = 0
print(human_readable_number(residual.sum()))

815.8M


변위 0의 오차

In [6]:
F = compute_nodal_forces(K, elements, torch.zeros([coords.shape[0], 3], dtype=torch.float64), device="cpu")
residual = torch.norm(F - F_target, dim=-1)
residual[rbe2] = 0
print(human_readable_number(residual.sum()))

35.6B


BiCongStab Solver

In [7]:
displacement = stable_conjugate_gradient_solver(K, elements, F_target, rbe2, u_init=outputs[:,0,:3], device="cpu", max_iter=100, tol=1e-6, dtype=torch.float64)

CG did not converge within the maximum number of iterations.


In [8]:
F = compute_nodal_forces(K, elements, displacement, dtype=torch.float64, device="cpu")
residual = torch.norm(F - F_target, dim=-1)
residual[rbe2] = 0
print(human_readable_number(residual.sum()))

86.1M


Modal

In [9]:
M = compute_c3d4_M_matrix(coords, elements, rho, device="cpu")
print(M.shape)

torch.Size([570111, 12, 12])


In [10]:
eig_value, eig_vector = vectorized_modal_solver(K, M, elements, rbe2, coords.shape[0], 5, max_iter=100, device="cpu", dtype=torch.float64)
print(eig_value, eig_vector.shape)

RuntimeError: unsupported operation: some elements of the input tensor and the written-to tensor refer to a single memory location. Please clone() the tensor before performing the operation.

하이브리드 테스트

In [11]:
e = identify_tetrahedral_shared_faces(elements, device="cpu")

In [12]:
element_edges = torch.cat((e[:,0,0].unsqueeze(-1), e[:,1,0].unsqueeze(-1)), dim=-1).squeeze(-1)

In [13]:
element_edges = torch.cat((e[:,0,0].unsqueeze(-1), e[:,1,0].unsqueeze(-1)), dim=-1).squeeze(-1)
F_e = F[elements]
R_e = rbe2_mask[elements]
print(elements.shape, element_edges.shape, F_e.shape, R_e.shape, K.shape)

torch.Size([570111, 4]) torch.Size([1097840, 2]) torch.Size([570111, 4, 3]) torch.Size([570111, 4]) torch.Size([570111, 12, 12])
